In [ ]:
import plotly
import numpy as np 
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt 
import os
import glob
from tqdm import tqdm
import yaml

In [ ]:
df=pd.read_pickle("result.pkl")
df

In [ ]:
#to determine if the MD simulation started from a checkpoint (save the cpt in that case) or outlier
for label in df.sys_label.unique():
    setup=yaml.safe_load(open(f"../md_run/{label}/setting.yml", 'r'))
    if setup['checkpoint']:
        df.loc[(df['sys_label'] == label),['cpt']]= setup['checkpoint'].split('/')[-2]  # sim label from which the cpt came
    elif 'md_run' in setup['pdb_file']:   # started from an outlier
        #print(setup['pdb_file'])
        df.loc[(df['sys_label'] == label),['outlier_MD_label']]=setup['pdb_file'].split('.pd')[0].split('/')[-1].split('_00')[0]
        df.loc[(df['sys_label'] == label),['outlier_MD_frame']]=int(setup['pdb_file'].split('.pd')[0].split('_')[-1])
    df.loc[(df['sys_label'] == label),['wall_clockt']]=int(label.split('_')[-1])  # to add column for starting wall clock times
print (df)

In [ ]:
df.outlier_MD_label.unique()

In [ ]:
print((df['pdb'].value_counts().to_list()))

In [ ]:
# to form md_steps array for the whole df
timeline = {}
md_step=[]
c=0
for gpu_id in df.gpu_id.unique(): 
    sub_df = df[df.gpu_id == gpu_id]
    i=0                      # simulation no. (starting from 0) for the particular GPU id
    for sys_label in sub_df.sys_label.unique(): 
        #current_time = int(sys_label.split('_')[-1]) - 1692329418     # mapping the wall clock starting time of simulation to the beginning of the main job
        #timeline[f"gpu_{gpu_id}"].append(current_time)
        t = list(map(lambda x: x + i*1000, sub_df[sub_df.sys_label == sys_label]['frame'].to_list()))
        c+=1
        #print(len(t),c)
        md_step+=t
        #sub_df.loc[sub_df.sys_label == sys_label]['md_step']=t.copy()
        i+=1

In [ ]:
print(len(md_step),max(md_step))

In [ ]:
len(df)

In [ ]:
df['md_step']=md_step.copy()
df

In [ ]:
# initial weights

df['weight']=1
df

In [ ]:
#changing the weights
print(int(len(df)/6))
mds=0     # will be used to iterate over md steps (multiples of 1000)
while mds < int(len(df)/6):
    for sys_label in df[df.md_step==mds].sys_label.unique():   # iterate over sys_labels at the same md_step
        #print(df[df.sys_label==sys_label])
        if df.loc[(df['sys_label']==sys_label) & (df['frame']==0)]['cpt'].to_list() and mds!=0: # if cpt exists: weight should be the same as that of the continued traj
            gpu_id=str(int(df.loc[(df.sys_label==sys_label) & (df.frame==0)]['gpu_id']))
            #print(gpu_id)
            df.loc[(df.sys_label==sys_label),['weight']]=float(df.loc[(df.gpu_id==gpu_id) & (df.md_step==mds-1)]['weight'])  # weight should be equal to the wt of frame at the previous md_step. 
            #print(df.loc[(df['gpu_id']==gpu_id) & (df['md_step']==mds-1)])
            
            
        # to reweight outlier trajectories
        if sys_label in df.outlier_MD_label.unique():
            sdf=df[(df.outlier_MD_label == sys_label) & (df.frame==0)].sort_values('md_step')   # which simulations had an outlier IC from this sys_label. Need to sort wrt md_step becoz reweighting should be done in the order of outlier detection
            for md_step in sdf.md_step.unique():        
                for outlier_MD_frame in sdf[sdf.md_step==md_step].outlier_MD_frame.unique():
                    fac=len(sdf[(sdf.md_step==md_step) & (sdf.outlier_MD_frame==outlier_MD_frame)])  # how many simulations started from the exact same outlier frame at this md_step 
                    print(fac)
                    df.loc[(df.sys_label==sys_label) & (df.frame >= outlier_MD_frame), ['weight']]/=(fac+1)  # reweight the parent trajectory
                    df.loc[(df.outlier_MD_label==sys_label) & (df.md_step >= md_step) & (df.md_step <= (md_step+999)), ['weight']]= float(df.loc[(df.sys_label==sys_label) & (df.frame == 999)]['weight'])  # reweight the child trajectory(ies)
            #print(sdf)
    #print(mds)
    mds+=1000      # move on to next set of simulations  
#df

                                                      pdb  \
939000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                      dcd  frame      rmsd  \
939000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.344074   

               Q                                         embeddings  \
939000  0.000001  [-0.08905251324176788, -0.6864911913871765, 0....   

        lof_score            sys_label gpu_id  cluster_label   wall_clockt  \
939000       -1.0  md_run_6_1697213153      6             95  1.697213e+09   

        cpt     outlier_MD_label  outlier_MD_frame  md_step  weight  
939000  NaN  md_run_6_1697181505              21.0    38000     1.0  
                                                       pdb  \
28000    /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
253000   /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
1154000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
709000   /project/zerze/ayushg/a

                                                      pdb  \
32000   /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
257000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
708000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                      dcd  frame      rmsd  \
32000   /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.717818   
257000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.661502   
708000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.729348   

               Q                                         embeddings  \
32000   0.003776  [-0.7215069532394409, -0.20375889539718628, 0....   
257000  0.007048  [-0.283432275056839, -0.40280699729919434, 0.5...   
708000  0.000203  [-0.08938691020011902, -0.1582384705543518, 0....   

        lof_score            sys_label gpu_id  cluster_label   wall_clockt  \
32000        -1.0  md_run_2_1697196926      2            241  1.697197e+09   
2

                                                      pdb  \
260000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
485000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
936000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                      dcd  frame      rmsd  \
260000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.579274   
485000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.331954   
936000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.099660   

                   Q                                         embeddings  \
260000  3.969745e-18  [0.651121973991394, -0.7338123917579651, 0.441...   
485000  5.934590e-17  [0.651121973991394, -0.7338123917579651, 0.441...   
936000  2.333504e-15  [0.738865077495575, -0.5471856594085693, 0.356...   

        lof_score            sys_label gpu_id  cluster_label   wall_clockt  \
260000       -1.0  md_run_3_1697204856      3            327  1

                                                      pdb  \
265000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                      dcd  frame      rmsd  \
265000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  7.966099   

               Q                                         embeddings  \
265000  0.000096  [0.33035796880722046, -0.42361417412757874, -0...   

        lof_score            sys_label gpu_id  cluster_label   wall_clockt  \
265000       -1.0  md_run_3_1697218047      3            421  1.697218e+09   

        cpt     outlier_MD_label  outlier_MD_frame  md_step  weight  
265000  NaN  md_run_7_1697194558             676.0    40000     1.0  
31000
                                                       pdb  \
67000    /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
1202000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                       dcd  frame      rmsd  \
67000   

                                                       pdb  \
36000    /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
261000   /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
486000   /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
937000   /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
1162000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
724000   /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                       dcd  frame      rmsd  \
36000    /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.546173   
261000   /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.470339   
486000   /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  5.739762   
937000   /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  5.754650   
1162000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.556364   
724000   /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  5.043646   

           

...

                                                       pdb  \
1299000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                       dcd  frame      rmsd  \
1299000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  5.970151   

                Q                                         embeddings  \
1299000  0.000003  [0.4890187084674835, 1.2444466352462769, -0.11...   

         lof_score            sys_label gpu_id  cluster_label   wall_clockt  \
1299000  -1.113809  md_run_7_1697575990      7            469  1.697576e+09   

         cpt     outlier_MD_label  outlier_MD_frame  md_step  weight  
1299000  NaN  md_run_6_1697488403             242.0   173000     1.0  
140000
141000
                                                       pdb  \
1292000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                       dcd  frame      rmsd  \
1292000  /project/zerze/ayushg/argonne/ddmd/rna8/tes

151000
152000
153000
                                                       pdb  \
1116000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                       dcd  frame      rmsd  \
1116000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.185325   

                Q                                         embeddings  \
1116000  0.185109  [-0.6563220024108887, 0.5979481339454651, -0.6...   

         lof_score            sys_label gpu_id  cluster_label   wall_clockt  \
1116000  -1.019884  md_run_6_1697691016      6            160  1.697691e+09   

         cpt     outlier_MD_label  outlier_MD_frame  md_step  weight  
1116000  NaN  md_run_6_1697526103             255.0   215000     1.0  
154000
155000
                                                      pdb  \
158000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                      dcd  frame      rmsd  \
158000  /project/zerze/ayushg/argo

                                                      pdb  \
878000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
653000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                      dcd  frame      rmsd  \
878000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.849244   
653000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.806799   

               Q                                         embeddings  \
878000  0.000003  [-0.8386213779449463, -0.5622714161872864, -0....   
653000  0.000022  [-1.00362229347229, -0.2681156396865845, 0.085...   

        lof_score            sys_label gpu_id  cluster_label   wall_clockt  \
878000       -1.0  md_run_5_1697653576      5            167  1.697654e+09   
653000       -1.0  md_run_4_1697655342      4            167  1.697655e+09   

        cpt     outlier_MD_label  outlier_MD_frame  md_step  weight  
878000  NaN  md_run_4_1697555701             367.0   202000   

173000
174000
                                                      pdb  \
667000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   
893000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                      dcd  frame      rmsd  \
667000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.630475   
893000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.459814   

               Q                                         embeddings  \
667000  0.000043  [0.34431949257850647, 0.37488752603530884, -0....   
893000  0.000048  [0.34431949257850647, 0.37488752603530884, -0....   

        lof_score            sys_label gpu_id  cluster_label   wall_clockt  \
667000       -1.0  md_run_4_1697693113      4            221  1.697693e+09   
893000       -1.0  md_run_5_1697693968      5            221  1.697694e+09   

        cpt     outlier_MD_label  outlier_MD_frame  md_step  weight  
667000  NaN  md_run_4_1697580002             463

192000
193000
                                                      pdb  \
886000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                      dcd  frame      rmsd  \
886000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...      0  6.920026   

               Q                                         embeddings  \
886000  0.239793  [-0.3366245627403259, 0.5633150339126587, -0.9...   

        lof_score            sys_label gpu_id  cluster_label   wall_clockt  \
886000        0.0  md_run_5_1697675147      5            288  1.697675e+09   

        cpt     outlier_MD_label  outlier_MD_frame  md_step  weight  
886000  NaN  md_run_6_1697634391             625.0   210000     1.0  
194000
                                                       pdb  \
1330000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m...   

                                                       dcd  frame      rmsd  \
1330000  /project/zerze/ayushg/argonne/ddmd/rna8/test/m

In [ ]:
print((df['weight'].value_counts()))

In [ ]:
df.to_pickle("reweighted.pkl")

In [ ]:
df.weight.unique()
